In [9]:
from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
import time
import requests
from selenium.webdriver.common.keys import Keys

In [28]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_0").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_0").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_0").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(2)

driver.find_element_by_id("btn_0").click()

daily_boxoffice_html = driver.page_source

driver.quit()

In [29]:
parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_0"})

movie_list = daily_boxoffice.find_all("tr")

In [30]:
import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 

In [31]:
daily_boxoffice = pd.DataFrame(daily_movies) 

In [32]:
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))

In [33]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0326(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

In [34]:
daily_boxoffice.to_sql("boxoffice_0326", con, chunksize=1000, if_exists='replace')

In [35]:
con.commit()

con.close()

In [36]:
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()


In [37]:
cursor.execute("SELECT * FROM boxoffice_0326")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0326", con, index_col='index')

df

,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 26일(화),돈,0,2019-03-20,"890,831,280",55.7%,"110,062","1,773,468","1,344","6,942"
1,2019년 03월 26일(화),캡틴 마블,0,2019-03-06,"249,453,760",15.6%,"29,769","5,295,488",806,"3,285"
2,2019년 03월 26일(화),악질경찰,0,2019-03-20,"135,373,220",8.5%,"16,565","224,861",691,"2,345"
3,2019년 03월 26일(화),우상,0,2019-03-20,"82,763,900",5.2%,"10,032","157,830",616,"1,700"
4,2019년 03월 26일(화),이스케이프 룸,0,2019-03-14,"52,724,540",3.3%,"6,261","557,876",421,945
5,2019년 03월 26일(화),더 길티,7,2019-03-27,"17,573,000",1.1%,"2,329","4,172",37,37
6,2019년 03월 26일(화),언더독,9,2019-01-16,"16,456,000",1.0%,"1,894","192,921",9,9
7,2019년 03월 26일(화),그린 북,0,2019-01-09,"10,502,010",0.7%,"1,593","412,978",61,104
8,2019년 03월 26일(화),라스트 미션,-2,2019-03-14,"10,424,550",0.7%,"1,465","96,598",105,171


In [41]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_114").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_114").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_114").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_114").click()

daily_boxoffice_html = driver.page_source

driver.quit()



In [44]:
import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_114"})

movie_list = daily_boxoffice.find_all("tr")

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    
daily_boxoffice = pd.DataFrame(daily_movies)     
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))



In [45]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0325(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

daily_boxoffice.to_sql("boxoffice_0325", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0325")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0325", con, index_col='index')

df


,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 25일(월),돈,0,2019-03-20,"1,042,451,680",59.0%,"126,999","1,663,406","1,330","6,948"
1,2019년 03월 25일(월),캡틴 마블,0,2019-03-06,"277,223,930",15.7%,"32,899","5,265,719",785,"3,330"
2,2019년 03월 25일(월),악질경찰,0,2019-03-20,"153,190,910",8.7%,"18,692","208,296",687,"2,381"
3,2019년 03월 25일(월),우상,0,2019-03-20,"81,839,880",4.6%,"9,918","147,798",590,"1,741"
4,2019년 03월 25일(월),이스케이프 룸,0,2019-03-14,"60,514,470",3.4%,"7,153","551,615",415,967
5,2019년 03월 25일(월),그대 이름은 장미,New,2019-01-16,"15,804,000",0.9%,"1,756","88,937",7,7
6,2019년 03월 25일(월),라스트 미션,1,2019-03-14,"12,252,050",0.7%,"1,656","95,133",110,178
7,2019년 03월 25일(월),그린 북,1,2019-01-09,"10,208,700",0.6%,"1,513","411,385",58,101
8,2019년 03월 25일(월),미래의 미라이,New,2019-01-16,"12,402,000",0.7%,"1,378","106,521",4,5


In [48]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_219").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_219").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_219").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_219").click()

daily_boxoffice_html = driver.page_source

driver.quit()



In [49]:
import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_219"})

movie_list = daily_boxoffice.find_all("tr")

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    
daily_boxoffice = pd.DataFrame(daily_movies)     
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))


In [50]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0324(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

daily_boxoffice.to_sql("boxoffice_0324", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0324")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0324", con, index_col='index')

df

,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 24일(일),돈,0,2019-03-20,"3,753,484,160",59.7%,"416,174","1,536,407","1,431","7,433"
1,2019년 03월 24일(일),캡틴 마블,0,2019-03-06,"1,275,765,220",20.3%,"142,380","5,232,820",947,"3,768"
2,2019년 03월 24일(일),악질경찰,0,2019-03-20,"427,941,590",6.8%,"46,589","189,604",718,"2,513"
3,2019년 03월 24일(일),우상,0,2019-03-20,"254,358,090",4.0%,"28,126","137,880",659,"1,884"
4,2019년 03월 24일(일),이스케이프 룸,0,2019-03-14,"214,363,930",3.4%,"23,494","544,462",491,"1,060"
5,2019년 03월 24일(일),정글북: 마법 원정대,0,2019-03-21,"90,106,890",1.4%,"11,520","22,534",372,495
6,2019년 03월 24일(일),페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,0,2019-03-21,"48,092,180",0.8%,"5,292","24,340",94,152
7,2019년 03월 24일(일),라스트 미션,1,2019-03-14,"27,805,210",0.4%,"3,345","93,477",101,169
8,2019년 03월 24일(일),그린 북,-1,2019-01-09,"23,715,060",0.4%,"3,302","409,872",62,99


In [51]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_338").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_338").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_338").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_338").click()

daily_boxoffice_html = driver.page_source

driver.quit()



In [52]:
import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_338"})

movie_list = daily_boxoffice.find_all("tr")

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    
daily_boxoffice = pd.DataFrame(daily_movies)     
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))


In [53]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0323(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

daily_boxoffice.to_sql("boxoffice_0323", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0323")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0323", con, index_col='index')

df

,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 23일(토),돈,0,2019-03-20,"4,347,878,286",59.0%,"480,715","1,120,233","1,410","7,486"
1,2019년 03월 23일(토),캡틴 마블,0,2019-03-06,"1,621,661,501",22.0%,"178,487","5,090,440",938,"3,821"
2,2019년 03월 23일(토),악질경찰,0,2019-03-20,"459,773,175",6.2%,"50,107","143,015",731,"2,676"
3,2019년 03월 23일(토),우상,0,2019-03-20,"294,901,025",4.0%,"32,698","109,754",675,"2,052"
4,2019년 03월 23일(토),이스케이프 룸,0,2019-03-14,"253,327,669",3.4%,"27,842","520,968",496,"1,067"
5,2019년 03월 23일(토),정글북: 마법 원정대,8,2019-03-21,"75,837,650",1.0%,"9,775","11,014",354,478
6,2019년 03월 23일(토),페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,-1,2019-03-21,"67,392,050",0.9%,"7,306","19,048",90,147
7,2019년 03월 23일(토),그린 북,2,2019-01-09,"27,736,410",0.4%,"3,802","406,570",57,89
8,2019년 03월 23일(토),라스트 미션,-1,2019-03-14,"32,349,160",0.4%,"3,801","90,132",103,172


In [54]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_453").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_453").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_453").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_453").click()

daily_boxoffice_html = driver.page_source

driver.quit()

import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_453"})

movie_list = daily_boxoffice.find_all("tr")

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    
daily_boxoffice = pd.DataFrame(daily_movies)     
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0322(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

daily_boxoffice.to_sql("boxoffice_0322", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0322")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0322", con, index_col='index')

df

,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 22일(금),돈,0,2019-03-20,"1,941,048,383",58.9%,"214,513","639,518","1,268","6,372"
1,2019년 03월 22일(금),캡틴 마블,0,2019-03-06,"609,926,536",18.5%,"65,373","4,911,953",812,"3,252"
2,2019년 03월 22일(금),악질경찰,0,2019-03-20,"247,429,553",7.5%,"27,478","92,908",772,"2,910"
3,2019년 03월 22일(금),우상,0,2019-03-20,"176,216,937",5.3%,"19,826","77,056",797,"2,566"
4,2019년 03월 22일(금),이스케이프 룸,0,2019-03-14,"116,031,904",3.5%,"12,564","493,126",447,"1,051"
5,2019년 03월 22일(금),페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,0,2019-03-21,"42,813,010",1.3%,"4,447","11,742",80,129
6,2019년 03월 22일(금),장난스런 키스,0,2019-03-27,"22,296,900",0.7%,"2,100","2,397",10,10
7,2019년 03월 22일(금),라스트 미션,1,2019-03-14,"16,801,460",0.5%,"2,032","86,331",114,189
8,2019년 03월 22일(금),양지의 그녀,-2,2019-03-21,"17,272,590",0.5%,"1,872","5,425",133,231


In [58]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_563").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_563").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_563").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_563").click()

daily_boxoffice_html = driver.page_source

driver.quit()



In [59]:
import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_563"})

movie_list = daily_boxoffice.find_all("tr")

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    
daily_boxoffice = pd.DataFrame(daily_movies)     
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0321(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

daily_boxoffice.to_sql("boxoffice_0321", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0321")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0321", con, index_col='index')

df

,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 21일(목),돈,0,2019-03-20,"1,407,813,290",56.2%,"170,825","425,005","1,166","5,802"
1,2019년 03월 21일(목),캡틴 마블,0,2019-03-06,"388,328,160",15.5%,"45,256","4,846,580",738,"2,923"
2,2019년 03월 21일(목),악질경찰,0,2019-03-20,"237,173,250",9.5%,"29,275","65,430",807,"3,106"
3,2019년 03월 21일(목),우상,0,2019-03-20,"165,600,800",6.6%,"20,549","57,230",849,"2,864"
4,2019년 03월 21일(목),이스케이프 룸,0,2019-03-14,"82,366,980",3.3%,"9,784","480,562",456,"1,050"
5,2019년 03월 21일(목),페이트 스테이 나이트 헤븐즈필 제2장 로스트 버터플라이,New,2019-03-21,"49,977,700",2.0%,"5,963","7,295",79,131
6,2019년 03월 21일(목),양지의 그녀,4,2019-03-21,"19,996,470",0.8%,"2,423","3,553",133,231
7,2019년 03월 21일(목),항거:유관순 이야기,1,2019-02-27,"13,573,330",0.5%,"2,021","1,141,084",69,87
8,2019년 03월 21일(목),라스트 미션,-2,2019-03-14,"14,330,100",0.6%,"1,915","84,299",111,188


In [60]:
path = 'chromedriver.exe' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.

driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do?loadEnd=0&searchType=search&sSearchFrom=2019-03-19&sSearchTo=2019-03-26&sMultiMovieYn=&sRepNationCd=&sWideAreaCd=")

driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

driver.find_element_by_id("btn_654").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_654").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_654").click() # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조

driver.implicitly_wait(4)

driver.find_element_by_id("btn_654").click()

daily_boxoffice_html = driver.page_source

driver.quit()

import pandas as pd

daily_movies = {'날짜': [],
'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_654"})

movie_list = daily_boxoffice.find_all("tr")

daily_movies['날짜'] = parsed_daily_boxoffice_html.find("div", {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).find_next_sibling("div",  {"class":"board_tit"}).text.strip() 

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    
daily_boxoffice = pd.DataFrame(daily_movies)     
def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice['순위 변동'] = daily_boxoffice['순위 변동'].map(lambda x: change_rank(x))
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice_0320(day date, title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")

daily_boxoffice.to_sql("boxoffice_0320", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()
import sqlite3
import pandas as pd

con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice_0320")

data = cursor.fetchall()

df = pd.read_sql("SELECT * FROM boxoffice_0320", con, index_col='index')

df

,날짜,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
index,,,,,,,,,,
0,2019년 03월 20일(수),돈,3,2019-03-20,"1,369,138,680",54.4%,"165,833","254,180","1,146","5,258"
1,2019년 03월 20일(수),캡틴 마블,-1,2019-03-06,"395,671,940",15.7%,"46,019","4,801,324",729,"2,840"
2,2019년 03월 20일(수),악질경찰,9,2019-03-20,"270,162,120",10.7%,"33,072","36,155",833,"3,059"
3,2019년 03월 20일(수),우상,4,2019-03-20,"222,873,850",8.9%,"27,175","36,681",900,"2,892"
4,2019년 03월 20일(수),이스케이프 룸,-3,2019-03-14,"101,006,850",4.0%,"12,025","470,778",513,"1,196"
5,2019년 03월 20일(수),1919 유관순,3,2019-03-14,"17,036,400",0.7%,"2,147","22,696",122,130
6,2019년 03월 20일(수),라스트 미션,-4,2019-03-14,"15,043,290",0.6%,"2,010","82,384",177,248
7,2019년 03월 20일(수),그린 북,3,2019-01-09,"9,440,050",0.4%,"1,378","399,921",48,86
8,2019년 03월 20일(수),항거:유관순 이야기,-3,2019-02-27,"7,866,480",0.3%,"1,243","1,139,063",106,119
